# Data cleaning


In the section below, the original dataset will be examined and cleaned up if necessary.  
The dataset is from the source below:

https://hogeschoolutrecht.sharepoint.com/:x:/r/sites/InstituteforICT-onderwijs/Gedeelde%20%20documenten/HBO-ICT%20-%20AAI/Data-driven%20business/sap_storing_data_hu_project.csv?d=w6af4a30998cb446283565c25459c2351&csf=1&web=1&e=bmoqi0

## Imports

In [1]:
#: Data analysis tools
import pandas as pd
import numpy as np
from scipy import stats

## Fetching

In [2]:
#: Requires a lot of calculation power.
df = pd.read_csv("../Data/CSV/sap_storing_data_hu_project.csv", low_memory=False)
oorzaakcodes = pd.read_csv('../Data/CSV/oorzaakcodes.csv', sep=';')
geocodes = pd.read_csv('../Data/CSV/geocodes.csv', sep=';')
print(f'No less than {df.shape[0]} rows and {df.shape[1]} columns!')

No less than 898526 rows and 140 columns!


## Cleaning

In [3]:
columns = {
    '#stm_sap_meldnr': 'meldnummer', 
    'stm_sap_meld_ddt': 'melddatum', 
    'stm_aanntpl_dd': 'startdatum_monteur', 
    'stm_aanntpl_tijd': 'starttijd_monteur', 
    'stm_fh_ddt': 'oplosdatum', 
    'stm_prioriteit': 'prioriteit', 
    'stm_oorz_code': 'oorzaakcode', 
    'stm_geo_gst': 'geocode', 
    'stm_sap_meldtekst_lang': 'eerste_beschrijving', 
    'stm_sap_meldtekst': 'tweede_beschrijving', 
    'stm_oorz_tekst_kort': 'derde_beschrijving', 
    'stm_fh_duur': 'duur'
}

df = df.rename(columns=columns)

In [4]:
#: Sort on priority from high to low and all dates from old to new.
df = df.sort_values(by=['prioriteit', 'melddatum'], ascending=False)

In [5]:
#: Delete all rows where repairtime is lower than 0 or bigger than 360 minutes.
df = df[df['duur'] != 0.0]
df = df[df['duur'] < 360.0]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 540712, amount of columns 140.


In [6]:
df['starttijd_monteur'] = df.starttijd_monteur.fillna('23:59')  #: Replace empty start times with '23:59'.
df['startdatum_monteur'] = df['startdatum_monteur'] + ' ' + df['starttijd_monteur'] #: Concat start date and time
df = df.drop(columns=['starttijd_monteur'])

In [7]:
#: This cell needs a lot of time as it is changing the data types for all the rows.
df['melddatum'] = pd.to_datetime(df['melddatum'], infer_datetime_format=True)
df['startdatum_monteur'] = pd.to_datetime(df['startdatum_monteur'], infer_datetime_format=True)
df['oplosdatum'] = pd.to_datetime(df['oplosdatum'], infer_datetime_format=True)
df['geocode'] = pd.to_numeric(df.geocode, errors='coerce')

In [8]:
#: Drop rows where there are nan values.
df = df.dropna(subset=['meldnummer', 'melddatum', 'startdatum_monteur', 'oplosdatum', 'prioriteit', 'oorzaakcode', 'geocode'])
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 413724, amount of columns 139.


In [9]:
#: Delete all rows with duplicate dates.
df = df[df.melddatum != df.startdatum_monteur]
df = df[df.melddatum != df.oplosdatum]
df = df[df.startdatum_monteur != df.oplosdatum]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 400124, amount of columns 139.


In [10]:
#: Delete all rows with illogical dates.
df = df[df.melddatum < df.startdatum_monteur]
df = df[df.melddatum < df.oplosdatum]
df = df[df.startdatum_monteur < df.oplosdatum]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 397767, amount of columns 139.


In [11]:
#: Filter duplicate meldnummers and keep the last.
df = df.drop_duplicates(subset=['meldnummer'], keep='last')
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 263140, amount of columns 139.


In [12]:
#: Add column with hersteltijd.
df['hersteltijd'] = (df.oplosdatum - df.startdatum_monteur) / pd.Timedelta(minutes=1)
df.hersteltijd = df.hersteltijd.astype(int)

In [13]:
#: Drop rows where hersteltijd is 0 or longer then 360.
df = df[df.hersteltijd <= 360]
df = df[df.hersteltijd > 0]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 260737, amount of columns 140.


In [14]:
#: Drop all rows with non existing geocodes.
lijst_geocodes = list(geocodes.geocode)
df = df[df['geocode'].isin(lijst_geocodes)]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 259196, amount of columns 140.


In [15]:
#: Drop all rows with non existing oorzaak codes.
lijst_oorzaakcodes = list(oorzaakcodes.Code)
df = df[df['oorzaakcode'].isin(lijst_oorzaakcodes)]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 259140, amount of columns 140.


In [16]:
#: Drop all rows with non existing prioriteiten.
lijst_prioriteit = range(1, 10)
df = df[df['prioriteit'].isin(lijst_prioriteit)]
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Amount of rows 259140, amount of columns 140.


The recovery times are defined per minute, this does not make the dataset transparent or easy to predict. For this reason, we have sorted the recovery times per fifteen minutes.

In [17]:
#: Create new column repairtimes 15 to represent the 15 minute interval of the original repair time.
df['hersteltijd_15'] = None
#: Iterate trough all repairtimes in intervals of 15 minutes.
for period in range(0, df['hersteltijd'].max(), 15):
    df['hersteltijd_15'] = np.where(df['hersteltijd'] > period, period, df['hersteltijd_15'])

In [18]:
#: Convert some numeric values to int
df = df.astype({ 'meldnummer': 'int32', 'prioriteit': 'int32', 'geocode': 'int32', 'oorzaakcode': 'int32', 'hersteltijd_15': 'int32' })

In [19]:
df['stm_arbeid'].sample(5)

602741     1.0
471691    50.0
459365    10.0
297847     3.0
363063     8.0
Name: stm_arbeid, dtype: float64

In [20]:
df.corrwith(df['hersteltijd'])

meldnummer                   0.004997
stm_mon_nr                   0.046049
stm_geo_mld_uit_functiepl    0.027174
stm_equipm_nr_mld            0.005202
stm_km_van_mld              -0.001165
stm_km_tot_mld               0.016265
prioriteit                  -0.002255
stm_mon_nr__statuscode      -0.031056
geocode                      0.026947
stm_geo_gst_uit_functiepl    0.026947
stm_equipm_nr_gst            0.001879
stm_km_van_gst              -0.000511
stm_km_tot_gst               0.011398
oorzaakcode                 -0.050509
stm_fh_status               -0.122163
stm_tao_telling_mutatie      0.024709
stm_contractgeb_mld         -0.019410
stm_contractgeb_gst         -0.014085
stm_arbeid                   0.792926
stm_progfh_gw_lwd_datum      0.064661
stm_progfh_gw_teller         0.103485
duur                         0.662829
stm_reactie_duur             0.015307
stm_status_sapnaarmon        0.007352
stm_schadenr                 0.015541
stm_rapportage_maand         0.002149
stm_rapporta

We see here that "stm_arbeid" has a very high correlation of 0.792926, unfortunately we can't use this because we don't know what it means. If we had to take a guess, we think that it is actual labor time that was needed for the repair, but you only know this afterwards of course. Furthermore we see "stm_progfh_gw_teller" at number 2 which is the number of times a description has been changed, it correlates a bit with the recovery time. The logic behind this would be that for more complex problems you modify the description several times to describe the newly discovered problem.

These columns don't correlate well so we go for our own logic, "geocode", "oorzaakcode" and "prioriteit". Geocode because the location can have a lot of influence on the repair, if it is in a busy city it will probably take longer than a quiet meadow. Oorzaakcode because a certain problem can take longer than another and prioriteit because something with a high priority or can be tackled faster or can take longer.

In [21]:
#: Reorder and select high correlation columns.
df = df[['meldnummer', 'melddatum', 'startdatum_monteur', 'oplosdatum', 'prioriteit', 'oorzaakcode', 'geocode', 
         'eerste_beschrijving', 'tweede_beschrijving', 'derde_beschrijving', 'hersteltijd', 'hersteltijd_15']]

In [22]:
print(f'Minimal repair time: {df["hersteltijd_15"].min()}, maximal repair time: {df["hersteltijd_15"].max()}')
df = df[(np.abs(stats.zscore(df[['hersteltijd_15']])) < 3).all(axis=1)]
print(f'Minimal repair time: {df["hersteltijd_15"].min()}, maximal repair time: {df["hersteltijd_15"].max()}')
print(f'Amount of rows {df.shape[0]}, amount of columns {df.shape[1]}.')

Minimal repair time: 0, maximal repair time: 345
Minimal repair time: 0, maximal repair time: 195
Amount of rows 252751, amount of columns 12.


Here we delete all the outliers to have a better dataset so we can predict values easier.

## Displaying/Checking

In [23]:
df.sample(5)

,meldnummer,melddatum,startdatum_monteur,oplosdatum,prioriteit,oorzaakcode,geocode,eerste_beschrijving,tweede_beschrijving,derde_beschrijving,hersteltijd,hersteltijd_15
221859,80351261,2006-03-07 10:57:10,2006-03-07 13:42:00,2006-03-07 14:01:00,2,215,49,Mo Sein 625 rood gedoofd Logboeknr RBV : 2...,Mo Sein 625 rood gedoofd,Lamp defect. Lamp lijkt optisch goed maa,19,15
746592,80838699,2016-02-24 09:37:34,2016-02-24 10:00:00,2016-02-24 13:06:00,2,151,506,"Amf : gasleiding geraakt thv wsl 657, graag di...","Amf : gasleiding geraakt thv wsl 657, gr",lekke gasleiding gebouwen,186,180
388868,80560373,2010-05-26 11:53:00,2010-05-26 12:25:00,2010-05-26 13:00:00,5,215,529,Bkl : Verlichting gedoofd van klok sp 2 naast...,Bkl : Verlichting gedoofd van klok sp 2,NaN,35,30
229711,80361263,2006-05-30 17:18:00,2006-05-30 17:57:00,2006-05-30 18:02:00,1,230,608,Apd wsl 67 RL nic. tobs..,Apd wsl 67 RL nic. tobs..,NaN,5,0
260656,80399620,2007-03-15 06:01:00,2007-03-15 08:30:00,2007-03-15 08:44:00,4,215,618,Ehv : SMC Telegyr 8020 Overvieuwsysteem lamp ...,Ehv : SMC Telegyr 8020 Overvieuwsysteem,NaN,14,0


In [24]:
df.columns

Index(['meldnummer', 'melddatum', 'startdatum_monteur', 'oplosdatum',
       'prioriteit', 'oorzaakcode', 'geocode', 'eerste_beschrijving',
       'tweede_beschrijving', 'derde_beschrijving', 'hersteltijd',
       'hersteltijd_15'],
      dtype='object')

In [25]:
df.dtypes

meldnummer                      int32
melddatum              datetime64[ns]
startdatum_monteur     datetime64[ns]
oplosdatum             datetime64[ns]
prioriteit                      int32
oorzaakcode                     int32
geocode                         int32
eerste_beschrijving            object
tweede_beschrijving            object
derde_beschrijving             object
hersteltijd                     int32
hersteltijd_15                  int32
dtype: object

In [26]:
pd.isnull(df).sum()

meldnummer                  0
melddatum                   0
startdatum_monteur          0
oplosdatum                  0
prioriteit                  0
oorzaakcode                 0
geocode                     0
eerste_beschrijving         6
tweede_beschrijving         0
derde_beschrijving     171874
hersteltijd                 0
hersteltijd_15              0
dtype: int64

## Creating a subset

In [27]:
subset_df = df.rename(columns={'eerste_beschrijving': 'beschrijving'}).head(10)
subset_df = subset_df[['meldnummer', 'oorzaakcode', 'prioriteit', 'melddatum', 'geocode', 'beschrijving', 'hersteltijd', 'hersteltijd_15']]

## Saving

In [28]:
#: Save cleaned version and a subset.
df.to_csv('../Data/CSV/cleaned.csv', sep=';', index=False)
subset_df.to_csv('../Data/CSV/subset.csv', sep=';', index=False)